In [ ]:
import sys
import os #os.chdir("dir with *.m files")
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import spikeextractors as se
import kachery as ka
ka.set_config(fr='default_readonly')

In [ ]:
# Load the entire spikeforest analysis results
# Note: this needs to be updated -- will do that later
SF = ka.load_object('sha1://1667383039f1170256e4e3445d46eff367e202e5/output.json', fr='default_readonly')

In [ ]:
# Example of looping through all of the sorting results:
sorting_results = SF['SortingResults']
for sr in sorting_results[:10]:
    print(f"{sr['studyName']} {sr['recordingName']} {sr['sorterName']} {sr['firings']}")
    firings_path = ka.load_file(sr['firings'])
    sorting = se.MdaSortingExtractor(firings_path)
    print(len(sorting.get_unit_ids()))

In [ ]:
# Pillow's CDM

import matlab.engine
from pyentropy import DiscreteSystem
eng = matlab.engine.start_matlab()

In [ ]:
def calculate_entropies(sorting, method, word_length, sampling_frequency):
    output = np.zeros((len(sorting.get_unit_ids())))
    
    sampF = sampling_frequency
    wLen = word_length
    unit_ids = sorting.get_unit_ids()
    spike_trains = [sorting.get_unit_spike_train(unit_id=uid) for uid in unit_ids]
    maxT = np.max([np.max(st) for st in spike_trains]) / sampling_frequency
    for ii, unit_id in enumerate(unit_ids):
        spk = sorting.get_unit_spike_train(unit_id=unit_id)
        binSize = 0.05 * sampF 
        bins = np.arange(0, maxT*sampF, binSize)
        spkCount,_ = np.histogram(spk, bins)
        spkCount[spkCount>1] = 1
            
        i = 0
        words = None
        while i<len(spkCount)-wLen:
            if words is None:
                words = spkCount[i:i+wLen]
                i += wLen #increase counter by wLen
            else:
                words = np.vstack((words,spkCount[i:i+wLen]))
                i += wLen
        response = np.zeros(words.shape[0]).astype(int)
        xm = np.max(spkCount)+1
        # todo: this line only applies to a subset of the methods
        sys = DiscreteSystem(words.T.astype(int), (wLen,xm), response, (1,1))
        if method == 'plugin':
            sys.calculate_entropies(method='plugin', calc=['HX'])
            output[ii] = sys.H_plugin['HX']
        elif method == 'pt':
            sys.calculate_entropies(method='pt', calc=['HX'])
            output[ii] = sys.H_pt['HX']
        elif method == 'qe':
            sys.calculate_entropies(method='qe', calc=['HX'])
            output[ii] = sys.H_qe['HX']
        elif method == 'CDM':
            output[ii] = eng.entropyCDM(matlab.double(words.tolist()), 0)
    return output

In [ ]:
firings_path = ka.load_file(sorting_results[0]['firings'])
sorting = se.MdaSortingExtractor(firings_path)
calculate_entropies(sorting, method='pt', word_length=5, sampling_frequency=30000)